In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 34944020
paper_name = 'jin_liu_2021' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table S4-The information of all deletion mutants tested in the genome-wide screen.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4115 x 8


In [7]:
original_data.head()

,Gene,ORF,Score1,p-Value 1,Score2,p-Value 2,Score3,p-Value 3
0,AAC1,YMR056C,-0.06060,0.00000,-0.03887,0.00000,-0.03915,0.00001
1,AAC3,YBR085W,-0.05033,0.00001,-0.03380,0.00000,0.02464,0.00000
2,AAD3,YCR107W,-0.04791,0.00001,-0.05779,0.00001,-0.06725,0.00001
3,AAD4,YDL243C,0.25415,0.00001,0.03606,0.00000,0.05654,0.00002
4,AAD6,YFL056C,0.05228,0.00001,0.06851,0.00002,0.07088,0.00005


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ Gene, ORF, Score1, p-Value 1, Score2, p-Value 2, Score3, p-Value 3, orf]
Index: []


In [12]:
original_data['data'] = original_data[['Score1','Score2','Score3']].mean(axis=1)

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(4080, 1)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [22074]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,22074
data_type,value
orf,
YAL002W,-0.859527
YAL004W,0.064010
YAL005C,-0.169750
YAL007C,-0.020010
YAL008W,0.068457


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22074
,data_type,value
gene_id,orf,
2,YAL002W,-0.859527
1863,YAL004W,0.064010
4,YAL005C,-0.169750
5,YAL007C,-0.020010
6,YAL008W,0.068457


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          22074          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.859527 -6.665747
1863    YAL004W  0.064010  0.487414
4       YAL005C -0.169750 -1.323151
5       YAL007C -0.020010 -0.163354
6       YAL008W  0.068457  0.521855

# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 34944020...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.33s/it]

Updating the data_modified_on field...
